In [1]:
import pandas as pd

STEP 1

- get frequency of words
- fusion wit the databses containing the translations

STEP 2

- front 


# Extracting & cleaning occurence / words with filter

**cleaning rapide avec scipy/nltk?**

- Filter with the words of the english language : http://www.mieliestronk.com/wordlist.html
- stopword, numbers, names, special charaters, remove foreign words
- split les mots en deux avec un ' au milieu

**Fusion with the dataset containing french & english words**

- how to avoid a infinite runtime ?? 
- tokenization : https://machinelearningmastery.com/clean-text-machine-learning-python/

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize


# StopWords
# nltk.download()
stop_words = stopwords.words('english')
# print(stop_words)


# Special characters
List_special = []
for caracter in '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~"' :
    List_special.append(caracter)
    
    
# List of english words
with open("english_language_lowercase.txt", 'r', encoding='UTF-8') as file:
    lines_english_language1 = file.readlines()
    lines_english_language = [line.rstrip() for line in lines_english_language1]

In [16]:
from tqdm import tqdm
filename = "enwiki-20190320-words-frequency.txt"

with open(filename, 'r', encoding='UTF-8') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in tqdm(lines) if line.split(" ")[0] in lines_english_language]

# new_list = [line.split(" ") for line in lines]
liste_words = [line.split(" ")[0] for line in lines]
liste_occurences = [line.split(" ")[1] for line in lines]



df = pd.DataFrame({"words" : liste_words,
                 "occurences" : liste_occurences})

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2184780/2184780 [35:53<00:00, 1014.47it/s]


NameError: name 'pd' is not defined

In [1]:
df.head()

NameError: name 'df' is not defined

In [134]:
# Remove stop words from the occurence dataframe 

for i in range(len(stop_words)):
    try :
        df.drop([(df[df["words"] == stop_words[i]]).index[0]], inplace=True)
    except :
        continue
        
df.reset_index(drop=True, inplace=True)
df_occu = df

len(df_occu)

df_occu.to_csv("cleaned_english_words_occurence.csv")

56708

# Preprocessing Vocabulary English/French

In [147]:
# vocabulary processing containing : english, french

#vocabulary = pd.read_excel("Vocabulary.xlsx")
vocabulary=pd.read_csv('data.csv')
#ici on tri en faisant du preprocessing + en enlevant les dupliqués exacts

def remove_punctuation(text):
    special_punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in text:
        if ele in special_punc:
            text = text.replace(ele, "")
    return text

def remove_stopwords(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = ' '.join([word for word in text_tokens if not word in stopwords.words()])
    return tokens_without_sw

def remove_empty(text):
    return " ".join(text.strip().split())

def preprocessing_lower_punc_sw_stem(df, col_name):
    
    #Preprocessing (" ",lower,punctuation,stop_words,stemming)
    df[col_name]=df[col_name].apply(lambda x: remove_empty(x))
    df[col_name]=df[col_name].apply(lambda x: x.lower())
    
    porter = PorterStemmer()
    df[col_name + '_nopunct']=df[col_name].apply(lambda x: remove_punctuation(x))
    df[col_name + '_noswords']=df[col_name + '_nopunct'].apply(lambda x: remove_stopwords(x))
    df[col_name + '_stem']=df[col_name + '_noswords'].apply(lambda x: porter.stem(x))

    #Duplicates
    df=df.drop_duplicates(subset=[col_name])
    df=df.drop_duplicates(subset=[col_name + '_stem'])
    df=df.reset_index(drop=True)
    
    
for col_name in ["ENGLISH", "FRENCH"]:
    preprocessing_lower_punc_sw_stem(vocabulary, col_name)
    
# vocabulary.to_csv("clean_vocabulary.csv")

In [149]:
vocabulary

,FRENCH,ENGLISH,THEME,IMPORTANT,ENGLISH_nopunct,ENGLISH_noswords,ENGLISH_stem,FRENCH_nopunct,FRENCH_noswords,FRENCH_stem
0,accepter,to accept,verbes a connaitre en,Non,to accept,accept,accept,accepter,accepter,accept
1,acheter,to buy,verbes a connaitre en,Non,to buy,buy,buy,acheter,acheter,achet
2,aider,to help,verbes a connaitre en,Non,to help,help,help,aider,aider,aider
3,aimer,"to like, to love",verbes a connaitre en,Non,to like to love,like love,like lov,aimer,aimer,aimer
4,ajouter,to add,verbes a connaitre en,Non,to add,add,add,ajouter,ajouter,ajout
...,...,...,...,...,...,...,...,...,...,...
14015,se rétablir de,"to get over stg, to recover from",covid coronavirus,NaN,to get over stg to recover from,get stg recover,get stg recov,se rétablir de,rétablir,rétablir
14016,soigner,to treat,covid coronavirus,NaN,to treat,treat,treat,soigner,soigner,soigner
14017,tester les patients,to test patients,covid coronavirus,NaN,to test patients,test patients,test pati,tester les patients,tester patients,tester pati
14018,tomber malade,to be taken ill,covid coronavirus,NaN,to be taken ill,taken,taken,tomber malade,tomber malade,tomber malad


In [156]:
# Deduplication avec BallTree

# from nltk.stem import LancasterStemmer
from sklearn.neighbors import BallTree
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#ici on essaye d'enlever les "almost duplicate" en vectorisant + utilisant BallTree sur les mots
#autrement dit on crée les colonnes "ID_Nearest_Neighbor" et "Distance_Nearest" sur un échantillon de 2000 lignes
sample_voc = vocabulary[0:2000]['ENGLISH_stem'].tolist()
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(sample_voc).todense()
matrix = pd.DataFrame(matrix, columns=vectorizer.get_feature_names_out ())
top_words = matrix.sum(axis=0).sort_values(ascending=False)

tree = BallTree(matrix.values, leaf_size=2)

distance_nearest, id_nearest = tree.query(
    matrix.values,
    k=3,)

df_voc2=vocabulary[0:2000]
df_voc2['Nearest_Neighbor']=df_voc2.iloc[id_nearest[:,1]]['ENGLISH'].tolist()
df_voc2['Proche_Voisin']=df_voc2.iloc[id_nearest[:,1]]['FRENCH'].tolist()
df_voc2['Distance_Nearest']=distance_nearest[:,1].tolist()
df_voc2['ID_Nearest_Neighbor']=id_nearest[:,1]

C:\Users\JOSEPH~1.PIC\AppData\Local\Temp/ipykernel_34092/3349348133.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_voc2['Nearest_Neighbor']=df_voc2.iloc[id_nearest[:,1]]['ENGLISH'].tolist()
C:\Users\JOSEPH~1.PIC\AppData\Local\Temp/ipykernel_34092/3349348133.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_voc2['Proche_Voisin']=df_voc2.iloc[id_nearest[:,1]]['FRENCH'].tolist()
C:\Users\JOSEPH~1.PIC\AppData\Local\Temp/ipykernel_34092/3349348133.py:25: SettingWithCopyWarning: 
A value is trying 

In [ ]:
# ici on delete les mots trop proche les uns des autres grâce au BallTree (dist<0.6)
print(len(df_voc2[df_voc2.Distance_Nearest<0.6]))
ids_todelete=df_voc2[df_voc2.Distance_Nearest<0.6].index.tolist()
ids_close=df_voc2[df_voc2.Distance_Nearest<0.6]['ID_Nearest_Neighbor'].tolist()
no_delete_ids=[]
for count,id_todelete in enumerate(ids_todelete):
    if id_todelete not in no_delete_ids:
        df_voc2.drop([id_todelete],inplace=True)
        no_delete_ids.append(ids_close[count])
    else: 
        pass
print(len(df_voc2[df_voc2.Distance_Nearest<0.6]))

# Fusion the two databases

In [162]:
vocabulary

,FRENCH,ENGLISH,THEME,IMPORTANT,ENGLISH_nopunct,ENGLISH_noswords,ENGLISH_stem,FRENCH_nopunct,FRENCH_noswords,FRENCH_stem
0,accepter,to accept,verbes a connaitre en,Non,to accept,accept,accept,accepter,accepter,accept
1,acheter,to buy,verbes a connaitre en,Non,to buy,buy,buy,acheter,acheter,achet
2,aider,to help,verbes a connaitre en,Non,to help,help,help,aider,aider,aider
3,aimer,"to like, to love",verbes a connaitre en,Non,to like to love,like love,like lov,aimer,aimer,aimer
4,ajouter,to add,verbes a connaitre en,Non,to add,add,add,ajouter,ajouter,ajout
...,...,...,...,...,...,...,...,...,...,...
14015,se rétablir de,"to get over stg, to recover from",covid coronavirus,NaN,to get over stg to recover from,get stg recover,get stg recov,se rétablir de,rétablir,rétablir
14016,soigner,to treat,covid coronavirus,NaN,to treat,treat,treat,soigner,soigner,soigner
14017,tester les patients,to test patients,covid coronavirus,NaN,to test patients,test patients,test pati,tester les patients,tester patients,tester pati
14018,tomber malade,to be taken ill,covid coronavirus,NaN,to be taken ill,taken,taken,tomber malade,tomber malade,tomber malad


In [190]:
cleaned_english_words_occurence = pd.read_csv("cleaned_english_words_occurence.csv", index_col=0)
cleaned_english_words_occurence

,words,occurences
0,also,5450043
1,first,4840311
2,one,4151416
3,new,4039430
4,two,3415323
...,...,...
56703,outstep,3
56704,rescans,3
56705,unbelieved,3
56706,unclasping,3


In [ ]:
sample_voc = vocabulary[0:2000]['ENGLISH_stem'].tolist()
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(sample_voc).todense()
matrix = pd.DataFrame(matrix, columns=vectorizer.get_feature_names_out ())
top_words = matrix.sum(axis=0).sort_values(ascending=False)

tree = BallTree(matrix.values, leaf_size=2)

distance_nearest, id_nearest = tree.query(
    matrix.values,
    k=3,)

df_voc2=vocabulary[0:2000]
df_voc2['Nearest_Neighbor']=df_voc2.iloc[id_nearest[:,1]]['ENGLISH'].tolist()
df_voc2['Proche_Voisin']=df_voc2.iloc[id_nearest[:,1]]['FRENCH'].tolist()
df_voc2['Distance_Nearest']=distance_nearest[:,1].tolist()
df_voc2['ID_Nearest_Neighbor']=id_nearest[:,1]

In [191]:
vocabulary.to_csv("clean_vocabulary.csv")

,words,occurences
0,also,5450043
1,first,4840311
2,one,4151416
3,new,4039430
4,two,3415323
...,...,...
56703,outstep,3
56704,rescans,3
56705,unbelieved,3
56706,unclasping,3


In [ ]:
cleaned_english_words_occurence

In [27]:
french_column = 'french'
english_column = 'english'
stem_colmun = 'stem'

df_french_english = pd.DataFrame(columns=[french_column, english_column, stem_colmun])



In [ ]:
df_french_english.loc[0]

In [28]:
df_french_english

,french,english,stem


In [ ]:
df.to_excel("english_frequency_list.xlsx")